In [12]:
from path import Path
import pandas as pd

from main import Worker

In [13]:
a = Path(r'.').glob(r'*.xlsx')
assert len(a) == 1

df_ = pd.read_excel(a[0], engine='openpyxl')

worker = Worker(df_)
df = worker.df

In [87]:
from dataclasses import dataclass
from typing import Tuple, List
from datetime import datetime
from dateutil import parser

def get_start_end_datetimes(row):
    def get_start_end_datetime(date_: datetime.date, start_: datetime.time, end_: datetime.time) -> Tuple:
        ret_start = datetime.combine(date_.date(), parser.parse(start_).time())
        ret_end = datetime.combine(date_.date(), parser.parse(end_).time())

        return ret_start, ret_end

    list_of_classes = []

    for cur_termin_no in range(1, 5):
        cur_termin_str = f'Termin {cur_termin_no}'
        cur_date = row[cur_termin_str]

        if cur_date is None:
            # print(f'{cur_termin_str} -> None!')
            continue

        start, end = row['Uhrzeit'].split(' - ')

        start_datetime, end_datetime = get_start_end_datetime(cur_date, start, end)

        list_of_classes.append((cur_termin_str, start_datetime, end_datetime))

    return list_of_classes

def get_subject(row: pd.Series) -> str:
    return row["Kurs"].replace(" -", "-").replace("- ", "-")

def get_description(row: pd.Series) -> str:
    names_only = row.drop(['Öffn.', 'Tag', 'Datum', 'Kurs', 'Uhrzeit', 'Termin 1', 'Termin 2', 'Termin 3', 'Termin 4'])
    list_of_trainers = names_only.dropna().to_list()
    list_of_trainers = sorted([n.capitalize() for n in list_of_trainers])

    return f'Potentially assigned trainers: {", ".join(list_of_trainers)}'

# ---

@dataclass(frozen=True)
class Termin:
    subject: str
    start: datetime
    end: datetime
    description: str

all_classes: [List[Termin]] = []
# row = df.iloc[79]
for idx, row in df.iterrows():
    cur_subject = get_subject(row)
    cur_list_of_subclasses = get_start_end_datetimes(row)
    cur_description = get_description(row)

    for sub_class_str, start_datetime, end_datetime in cur_list_of_subclasses:
        all_classes.append(Termin(
            subject=f'{cur_subject} - {sub_class_str}',
            start=start_datetime,
            end=end_datetime,
            description=cur_description
        ))

all_classes = [c for c in all_classes if 'Frank' in c.description and 'Anne' in c.description]

for c in all_classes:
    print(f'class   : {c.subject}')
    print(f'start   : {c.start}')
    print(f'end     : {c.end}')
    print(f'desc    : {c.description}')
    print()

print(f'len(all_classes): {len(all_classes)}')

class   : Einstiegs-Kurs - Termin 1
start   : 2022-01-05 18:30:00
end     : 2022-01-05 21:30:00
desc    : Potentially assigned trainers: Anne, Claudia, Frank, Orsi, Sosa

class   : Einstiegs-Kurs - Termin 2
start   : 2022-01-12 18:30:00
end     : 2022-01-12 21:30:00
desc    : Potentially assigned trainers: Anne, Claudia, Frank, Orsi, Sosa

class   : Offener Klettertreff - Termin 1
start   : 2022-01-25 19:00:00
end     : 2022-01-25 21:00:00
desc    : Potentially assigned trainers: Anne, Frank

class   : Offener Klettertreff - Termin 1
start   : 2022-02-08 19:00:00
end     : 2022-02-08 21:00:00
desc    : Potentially assigned trainers: Anne, Frank

class   : Offener Klettertreff - Termin 1
start   : 2022-03-29 19:00:00
end     : 2022-03-29 21:00:00
desc    : Potentially assigned trainers: Anne, Frank

len(all_classes): 5


                           subject               start                 end  \
0        Einstiegs-Kurs - Termin 1 2022-01-05 18:30:00 2022-01-05 21:30:00   
1        Einstiegs-Kurs - Termin 2 2022-01-12 18:30:00 2022-01-12 21:30:00   
2  Offener Klettertreff - Termin 1 2022-01-25 19:00:00 2022-01-25 21:00:00   
3  Offener Klettertreff - Termin 1 2022-02-08 19:00:00 2022-02-08 21:00:00   
4  Offener Klettertreff - Termin 1 2022-03-29 19:00:00 2022-03-29 21:00:00   

                                         description  
0  Potentially assigned trainers: Anne, Claudia, ...  
1  Potentially assigned trainers: Anne, Claudia, ...  
2         Potentially assigned trainers: Anne, Frank  
3         Potentially assigned trainers: Anne, Frank  
4         Potentially assigned trainers: Anne, Frank  
